In [ ]:
import numpy as np
import pandas as pd
import math
from google.colab import files
from scipy.spatial.distance import directed_hausdorff
from scipy import stats
from scipy.special import expit

uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c nfl-big-data-bowl-2024
!unzip nfl-big-data-bowl-2024.zip

100% 279M/279M [00:10<00:00, 32.2MB/s]
100% 279M/279M [00:10<00:00, 28.9MB/s]
Archive:  nfl-big-data-bowl-2024.zip
  inflating: games.csv               
  inflating: players.csv             
  inflating: plays.csv               
  inflating: tackles.csv             
  inflating: tracking_week_1.csv     
  inflating: tracking_week_2.csv     
  inflating: tracking_week_3.csv     
  inflating: tracking_week_4.csv     
  inflating: tracking_week_5.csv     
  inflating: tracking_week_6.csv     
  inflating: tracking_week_7.csv     
  inflating: tracking_week_8.csv     
  inflating: tracking_week_9.csv     


In [ ]:
# Initialize Data
players=pd.read_csv('players.csv')
# week1=pd.read_csv('tracking_week_1.csv')
# week2=pd.read_csv('tracking_week_2.csv')
week3=pd.read_csv('tracking_week_3.csv')
# week4=pd.read_csv('tracking_week_4.csv')
# week5=pd.read_csv('tracking_week_5.csv')
# week6=pd.read_csv('tracking_week_6.csv')
# week7=pd.read_csv('tracking_week_7.csv')
# week8=pd.read_csv('tracking_week_8.csv')
# week9=pd.read_csv('tracking_week_9.csv')
games=pd.read_csv('games.csv')
plays=pd.read_csv('plays.csv')
tackles = pd.read_csv('tackles.csv')

In [ ]:
def extract_data_sets(data, gameId, playId, frameId):
    wg = data[(data['gameId'] == gameId)].copy()
    wgp = wg[(wg['playId'] == playId)].copy()
    wgpf = wgp[(wgp['frameId'] == frameId)].copy()
    wgpf_time = wgpf['time'].iloc[0]
    return wg, wgp, wgpf, wgpf_time

def processToVisualizeFrame(df1, df2, df3, df4, df5, gameId, playId, frameId):
    # Filter DataFrames directly during merge operations for efficiency
    df = df1[(df1['gameId'] == gameId) & (df1['playId'] == playId) & (df1['frameId'] == frameId)].copy()

    # Select only necessary columns in df4 for merging
    df4 = df4.drop(columns=['displayName'])

    # Merge DataFrames efficiently by specifying columns
    df = pd.merge(df, df2, on='gameId', how='left')
    df = pd.merge(df, df3, on=['playId', 'gameId'], how='left')
    df = pd.merge(df, df4, on='nflId', how='left')
    df = pd.merge(df, df5, on=['playId', 'gameId', 'nflId'], how='left')

    # Apply transformations to specific columns without conditional selection
    df['x'] = np.where(df['playDirection'] == 'right', 120 - df['x'], df['x'])
    df['y'] = np.where(df['playDirection'] == 'right', 53.3 - df['y'], df['y'])
    df['dir'] = (df['dir'] + 180) % 360
    df['o'] = (df['o'] + 180) % 360

    # Convert angles to radians and calculate speed components
    df['rDir'] = np.deg2rad(df['dir'].astype(float))
    df['xComponent'] = np.cos(df['rDir'])
    df['yComponent'] = np.sin(df['rDir'])
    df['xSpeed'] = df['xComponent'] * df['s']
    df['ySpeed'] = df['yComponent'] * df['s']

    # Calculate distances between players and the ball carrier efficiently
    ball_carrier = df[df['displayName'] == df['ballCarrierDisplayName']]
    bcx, bcy = ball_carrier['x'].iloc[0], ball_carrier['y'].iloc[0]
    df['xDist_to_bc'] = bcx - df['x']
    df['yDist_to_bc'] = bcy - df['y']
    df['dist_to_bc'] = np.sqrt(df['xDist_to_bc']**2 + df['yDist_to_bc']**2)

    return df

def calculate_path_weights(pdf, start, node):
    x_over_y = (node[0] - start[0]) / (node[1] - start[1])
    iterations = abs(round(node[1] - start[1] * 10))
    path_weights = []

    # Precompute x and y coordinates for the entire range
    x_coordinates = start[0] * 10 - np.arange(iterations) * x_over_y
    y_coordinates = np.round(start[1] * 10) - np.arange(iterations)

    # Calculate x_floor, x_ceil, and y_val using arrays
    x_floor = np.floor(x_coordinates) - 10
    x_ceil = np.ceil(x_coordinates) + 10
    y_val = y_coordinates.astype(int)

    # Compute path weights using vectorized operations
    for x_floor_i, x_ceil_i, y_val_i in zip(x_floor, x_ceil, y_val):
        path_weights.append(np.sum(pdf[int(x_floor_i):int(x_ceil_i), y_val_i]))

    return np.sum(path_weights) / 5

def adjust_distance(distance, path_weight, start, end):
    adjusted_distance = distance * expit(path_weight)
    return float(adjusted_distance)

def create_Graph(start, df, pdf):
    graph = {}
    next_nodes = []

    if not df.empty:
        y = df['x'].max()
        next_nodes = [(x / 100, y) for x in range(15, 5315, 100)]

    for node in next_nodes:
        distance = np.linalg.norm(np.array(start) - np.array(node))

        if isinstance(pdf, float):
            adjusted_distance = distance
        else:
            path_weight = calculate_path_weights(pdf, start, node) / distance
            adjusted_distance = adjust_distance(distance, path_weight, start, node)

        graph[adjusted_distance] = [start, node]

    return graph


def get_Ideal_Path(start, graph):
    best_cost = min(graph)
    best_path_edges = graph[best_cost]
    return best_path_edges

def create_bc_pdf(df, locations):
    # Rotation Matrix
    xComponent, yComponent = df['xComponent'].values[0], df['yComponent'].values[0]
    r_Matrix = np.array([[xComponent, -yComponent], [yComponent, xComponent]])

    # Scaling Matrix
    speed_Ratio = (df['s'].values[0] ** 2) / (13.25 ** 2)
    bc_x = df['y'].values[0] + df['xSpeed'].values[0] * 0.5
    bc_y = df['x'].values[0] + df['ySpeed'].values[0] * 0.5

    dist = 2
    topLeftSMatrix = (dist + dist * speed_Ratio) * 0.5
    botRightSMatrix = (dist - dist * speed_Ratio) * 0.5
    s_Matrix = np.array([[topLeftSMatrix + 0.000001, 0], [0, botRightSMatrix - 0.000001]])

    # Covariance Matrix Calculations
    covariance_matrix = r_Matrix @ s_Matrix @ s_Matrix.T @ np.linalg.inv(r_Matrix)

    # Mu Values
    mu_val_x = df['y'].values[0] + df['xSpeed'].values[0] * 0.2
    mu_val_y = df['x'].values[0] + df['ySpeed'].values[0] * 0.2
    bc_mu = np.array([mu_val_x, mu_val_y])

    # Creating BallCarrier PDF
    bc_pdf = stats.multivariate_normal.pdf(locations, mean=bc_mu, cov=covariance_matrix)

    return bc_pdf

def create_pdf(df, locations, bc, graph=False):
    defensive_pdf, offensive_pdf = 0.0, 0.0
    defenders_pdfs = {}

    for index, row in df.iterrows():
        if row['displayName'] == row['ballCarrierDisplayName'] or row['displayName'] == 'football':
            continue

        # Rotation Matrix
        xComponent, yComponent = row['xComponent'], row['yComponent']
        r_Matrix = np.array([[xComponent, -yComponent], [yComponent, xComponent]])
        i_r_Matrix = np.linalg.inv(r_Matrix)

        # Scaling Matrix
        speed_Ratio = (row['s'] ** 2) / (13.25 ** 2)
        topLeftSMatrix = 0.5 + (row['dist_to_bc'] + row['dist_to_bc'] * speed_Ratio) * 0.5
        botRightSMatrix = 0.5 + (row['dist_to_bc'] - row['dist_to_bc'] * speed_Ratio) * 0.5
        s_Matrix = np.array([[topLeftSMatrix + 0.000001, 0], [0, botRightSMatrix - 0.000001]])

        # Covariance Matrix Calculations
        rs_Matrix = np.dot(r_Matrix, s_Matrix)
        rss_Matrix = np.dot(rs_Matrix, s_Matrix)
        covariance_Matrix = np.dot(rss_Matrix, i_r_Matrix)

        # Mu Values
        mu_val_x = row['y'] + row['xSpeed'] * 0.5
        mu_val_y = row['x'] + row['ySpeed'] * 0.5
        mu = [mu_val_x, mu_val_y]

        player_pdf = stats.multivariate_normal(mu, covariance_Matrix).pdf(locations)

        if row['club'] == row['defensiveTeam']:
            if not graph:
                defenders_pdfs[row['displayName']] = player_pdf
            defensive_pdf += player_pdf
        elif row['club'] == row['possessionTeam']:
            closest_tackler_distance = min(df.loc[df['club'] == df['defensiveTeam']]['dist_to_bc'])

            closest_tackler_distance_blocker = 13105
            yCoordinatePassRusher = 0
            xCoordinatePassRusher = 0
            speedPassRusher = 0

            df_defensive = df.loc[df['club'] == df['defensiveTeam']]
            for _, defense_row in df_defensive.iterrows():
                distance = np.sqrt((defense_row['x'] - row['x']) ** 2 + (defense_row['y'] - row['y']) ** 2)
                if distance <= closest_tackler_distance_blocker:
                    closest_tackler_distance_blocker = distance
                    xCoordinatePassRusher = defense_row['x']
                    yCoordinatePassRusher = defense_row['y']
                    speedPassRusher = defense_row['s']

            p12 = row['dist_to_bc']
            p23 = np.sqrt((bc['x'].values[0] - xCoordinatePassRusher) ** 2 + (bc['y'].values[0] - yCoordinatePassRusher) ** 2)
            p13 = np.sqrt((row['x'] - xCoordinatePassRusher) ** 2 + (row['y'] - yCoordinatePassRusher) ** 2)

            angleBetweenThreePlayers = np.arccos(((p12 ** 2) + (p13 ** 2) - (p23 ** 2)) / (2 * p12 * p13))
            degreesAngleBetweenThreePlayers = np.degrees(angleBetweenThreePlayers)
            angleFrom180 = abs(degreesAngleBetweenThreePlayers - 180)

            player_pdf = degreesAngleBetweenThreePlayers * (player_pdf / 180)
            offensive_pdf -= player_pdf

    if not graph:
        return defenders_pdfs, defensive_pdf, offensive_pdf
    else:
        return defensive_pdf, offensive_pdf

import numpy as np

def scale_path(segment):
    # Calculate the vector components
    dx = segment[1][0] - segment[0][0]
    dy = segment[1][1] - segment[0][1]

    # Calculate the length of the segment squared (avoiding square root operation)
    length_sq = dx ** 2 + dy ** 2

    # If the segment length squared is not zero, scale to 2.5 yards
    if length_sq != 0:
        scale_factor = 2.5 / np.sqrt(length_sq)
        scaled_dx = dx * scale_factor
        scaled_dy = dy * scale_factor
        new_segment = (segment[0], (segment[0][0] + scaled_dx, segment[0][1] + scaled_dy))
        return new_segment
    else:
        return segment  # Return the original segment if length is zero

def process(df, pdf):
    bc = df.loc[df['displayName'] == df['ballCarrierDisplayName']]
    dfDefensive = df.loc[(df['club']== df['defensiveTeam']) & (bc['x'].values[0] > (df['x'])) & ((df['x']) > 10)]

    start = tuple(bc[['y','x']].values[0])

    if start[1] <= 10:
        return None

    graph = create_Graph(start, dfDefensive, pdf)
    ideal_path = get_Ideal_Path(start, graph)
    return ideal_path

def process_frame(df, start_frame):
    bc = df.loc[df['displayName'] == df['ballCarrierDisplayName']]
    df_players = df.loc[bc['x'].values[0] + 2.5 > df['x']]

    x, y = np.mgrid[0:53.3:0.1, 0:120:0.1]
    locations = np.dstack((x, y))

    bc_pdf = create_bc_pdf(bc, locations)

    defenders_pdf, defensive_pdf, offensive_pdf = create_pdf(df_players, locations, bc)

    combined_pdf = offensive_pdf + defensive_pdf

    pressure_pdf = combined_pdf * bc_pdf

    path_deviation = 0

    modified_df = df[['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'time', 'jerseyNumber', 'club', 'event', 'possessionTeam', 'defensiveTeam', 'prePenaltyPlayResult', 'prePenaltyPlayResult']].copy()

    if df['frameId'].values[0] >= start_frame:
        ideal_path = process(df_players, combined_pdf)
        if ideal_path is None:
            path_deviation = 0
        else:
            start = (bc['y'].values[0], bc['x'].values[0])
            end = (np.round(start[0] + bc['xSpeed'].values[0] * 0.5, 3), np.round(start[1] + bc['ySpeed'].values[0] * 0.5, 3))
            actual_path = [start, end]
            norm_actual_path = scale_path(actual_path)
            norm_ideal_path = scale_path(ideal_path)
            path_deviation = directed_hausdorff(norm_actual_path, norm_ideal_path)[0]

    index = 0
    for displayName, pdf in defenders_pdf.items():
        modified_df.loc[((modified_df['displayName'] == displayName)), 'individual_defensive_pressure'] = np.sum(pdf * bc_pdf) / 5
        index += 1
    total_defensive_pressures = np.sum(defensive_pdf * bc_pdf) / 5
    total_pressures = [np.sum(pressure_pdf) / 5] * 23
    path_deviations = [path_deviation] * 23

    modified_df['total_defensive_pressures'] = [total_defensive_pressures] * 23
    modified_df['total_pressures'] = total_pressures
    modified_df['path_deviations'] = path_deviations

    return modified_df

def process_play(df1, df2, df3, df4, df5, playId, gameId):
    distinct_frames = list(df1.loc[(df1['playId'] == playId) & (df1['gameId'] == gameId), 'frameId'].unique())
    start_frame = df1.loc[(df1['playId'] == playId) & (df1['gameId'] == gameId) &
                          ((df1['event'] == 'handoff') | (df1['event'] == 'pass_outcome_caught') |
                           (df1['event'] == 'run')), 'frameId'].values[0]
    end_frame = df1.loc[(df1['playId'] == playId) & (df1['gameId'] == gameId) &
                        ((df1['event'] == 'tackle') | (df1['event'] == 'out_of_bounds') |
                         (df1['event'] == 'qb_slide')), 'frameId'].values[0]

    frame_dfs = []
    skipped_frames = []

    for frameId in distinct_frames:
        try:
            df = processToVisualizeFrame(df1, df2, df3, df4, df5, gameId, playId, frameId).copy()
            mod_df = process_frame(df, start_frame)
            frame_dfs.append(mod_df)
        except:
            print(f"Did not include {gameId} {playId} {frameId}")
            skipped_frames.append(gameId, playId, frameId)
            pass

    play_df = pd.concat(frame_dfs, axis=0, ignore_index=True)
    play_df['individual_pressure_percentage'] = play_df['individual_defensive_pressure'] / play_df['total_defensive_pressures']
    play_df['individual_pressure'] = play_df['individual_pressure_percentage'] * play_df['total_pressures']
    play_df['generated_path_deviation'] = play_df['individual_pressure_percentage'] * play_df['path_deviations']

    zero_total_pressures = play_df['total_pressures'] < 0
    columns_to_zero = ['individual_pressure_percentage','individual_pressure','generated_path_deviation']
    play_df.loc[zero_total_pressures, columns_to_zero] = 0

    play_df['start_frame'] = start_frame
    play_df['end_frame'] = end_frame

    return play_df, skipped_frames

def process_game(df1, df2, df3, df4, df5, gameId):
    game = df1[df1['gameId']==gameId]
    distinct_plays = list(game.playId.unique())

    play_dfs = []
    skipped_frames = []
    skipped_plays = []

    for playId in distinct_plays:
        try:
            play_df, temp = process_play(df1, df2, df3, df4, df5, playId, gameId)
            play_dfs.append(play_df)

            skipped_frames.extend(temp)
            print(f"Finished processing play{playId}   Progress({1+distinct_plays.index(playId)}/{len(distinct_plays)})")
        except:
            skipped_plays.append((gameId, playId))
            print(f"Did not include {gameId} {playId}")
    game_df = pd.concat(play_dfs, axis=0, ignore_index=True)

    return game_df, skipped_frames, skipped_plays

In [ ]:
week_games = list(week3.gameId.unique())

df = week3
print(df.shape[0])
for game in week_games:
    df_game = df[(df.gameId==game)] # Create Game DataFrame

    # Filter
    broken_playIds = list(df_game.loc[(df_game.event=='lateral'), 'playId'].unique()) # Get Broken Plays
    df_game = df_game[~df_game['playId'].isin(broken_playIds)] # Remove Broken Plays

    playIds = list(df_game.playId.unique()) # Prepare Play Iteration
    for play in playIds:
        df_play = df_game[(df_game.playId==play)].copy() # Create Game DataFrame
        frames = list(df_play.frameId.unique()) # Prepare Frame Iteration
        try:
            start_frame = df_play.loc[((df_play.event=='run') | (df_play.event=='handoff') | (df_play.event=='pass_outcome_caught')), 'frameId'].values[0]
        except:
            df = df[~(df['playId']==play)]
        try:
            end_frame = df_play.loc[((df_play.event=='tackle') | (df_play.event=='out_of_bounds') | (df_play.event=='qb_slide')), 'frameId'].values[0]
        except:
            df = df[~(df['playId']==play)]
        broken_frames = []
        for frame in frames:
            if df_play[df_play.frameId==frame].__len__() != 23:
                broken_frames.append(frame)
        df_play = df_play[~df_play['frameId'].isin(broken_frames)] # Remove Broken Plays
print(df.shape[0])

1415788
1344419


In [ ]:
week = 'week3'
dfs_processed, skipped_frames, skipped_plays = [], [], []
for gameId in week_games:
    df_temp, temp_skipped_frames, temp_skipped_plays = process_game(df, games, plays, players, tackles, gameId)
    try:
        dfs_processed.append(df_temp), skipped_frames.extend(temp_skipped_frames), skipped_plays.extend(temp_skipped_plays)
    except:
        print('did not add')

Finished processing play56   Progress(1/94)
Finished processing play84   Progress(2/94)
Finished processing play127   Progress(3/94)
Finished processing play190   Progress(4/94)
Finished processing play214   Progress(5/94)
Finished processing play240   Progress(6/94)
Finished processing play302   Progress(7/94)
Finished processing play419   Progress(8/94)
Finished processing play440   Progress(9/94)
Finished processing play465   Progress(10/94)
Finished processing play489   Progress(11/94)
Finished processing play534   Progress(12/94)
Finished processing play577   Progress(13/94)
Finished processing play601   Progress(14/94)
Finished processing play622   Progress(15/94)
Finished processing play643   Progress(16/94)
Finished processing play722   Progress(17/94)
Finished processing play746   Progress(18/94)
Finished processing play770   Progress(19/94)
Finished processing play791   Progress(20/94)
Finished processing play847   Progress(21/94)
Finished processing play958   Progress(22/94)

<ipython-input-4-2f127fb6a7f3>:178: RuntimeWarning: invalid value encountered in arccos
  angleBetweenThreePlayers = np.arccos(((p12 ** 2) + (p13 ** 2) - (p23 ** 2)) / (2 * p12 * p13))


Finished processing play2613   Progress(60/94)
Finished processing play2635   Progress(61/94)
Finished processing play2674   Progress(62/94)
Finished processing play2695   Progress(63/94)
Finished processing play2716   Progress(64/94)


<ipython-input-4-2f127fb6a7f3>:178: RuntimeWarning: invalid value encountered in arccos
  angleBetweenThreePlayers = np.arccos(((p12 ** 2) + (p13 ** 2) - (p23 ** 2)) / (2 * p12 * p13))


Finished processing play2740   Progress(65/94)
Finished processing play2761   Progress(66/94)
Finished processing play2785   Progress(67/94)
Finished processing play2924   Progress(68/94)
Finished processing play2945   Progress(69/94)
Finished processing play3007   Progress(70/94)
Finished processing play3030   Progress(71/94)
Did not include 2022092200 3053 13
Did not include 2022092200 3053
Finished processing play3093   Progress(73/94)
Finished processing play3114   Progress(74/94)
Finished processing play3135   Progress(75/94)
Finished processing play3156   Progress(76/94)
Finished processing play3179   Progress(77/94)
Finished processing play3200   Progress(78/94)
Finished processing play3221   Progress(79/94)
Finished processing play3308   Progress(80/94)
Finished processing play3439   Progress(81/94)
Finished processing play3460   Progress(82/94)
Finished processing play3484   Progress(83/94)
Finished processing play3505   Progress(84/94)
Finished processing play3569   Progress(

...

<ipython-input-4-2f127fb6a7f3>:178: RuntimeWarning: invalid value encountered in arccos
  angleBetweenThreePlayers = np.arccos(((p12 ** 2) + (p13 ** 2) - (p23 ** 2)) / (2 * p12 * p13))


Finished processing play2990   Progress(68/87)
Finished processing play3022   Progress(69/87)
Finished processing play3080   Progress(70/87)
Finished processing play3104   Progress(71/87)
Finished processing play3158   Progress(72/87)
Finished processing play3238   Progress(73/87)
Finished processing play3308   Progress(74/87)
Finished processing play3329   Progress(75/87)
Finished processing play3360   Progress(76/87)
Finished processing play3381   Progress(77/87)
Finished processing play3415   Progress(78/87)
Finished processing play3530   Progress(79/87)
Finished processing play3554   Progress(80/87)
Finished processing play3578   Progress(81/87)
Finished processing play3602   Progress(82/87)
Finished processing play3626   Progress(83/87)
Finished processing play3679   Progress(84/87)
Finished processing play3708   Progress(85/87)
Finished processing play3732   Progress(86/87)
Finished processing play3811   Progress(87/87)
Finished processing play65   Progress(1/82)
Finished process

<ipython-input-4-2f127fb6a7f3>:178: RuntimeWarning: invalid value encountered in arccos
  angleBetweenThreePlayers = np.arccos(((p12 ** 2) + (p13 ** 2) - (p23 ** 2)) / (2 * p12 * p13))


Finished processing play1181   Progress(28/82)
Finished processing play1213   Progress(29/82)
Finished processing play1259   Progress(30/82)
Finished processing play1363   Progress(31/82)
Finished processing play1384   Progress(32/82)
Finished processing play1413   Progress(33/82)
Finished processing play1434   Progress(34/82)
Finished processing play1460   Progress(35/82)
Did not include 2022092513 1484 56
Did not include 2022092513 1484
Finished processing play1505   Progress(37/82)
Finished processing play1553   Progress(38/82)
Finished processing play1634   Progress(39/82)
Finished processing play1655   Progress(40/82)
Finished processing play1683   Progress(41/82)
Finished processing play1769   Progress(42/82)
Finished processing play1855   Progress(43/82)
Finished processing play1892   Progress(44/82)
Finished processing play1964   Progress(45/82)
Finished processing play1991   Progress(46/82)
Finished processing play2062   Progress(47/82)
Finished processing play2160   Progress(

AttributeError: 'list' object has no attribute 'to_csv'

In [ ]:
final = pd.concat(dfs_processed, axis=0, ignore_index=True)
final.to_csv(f"{week}.csv", index=False)
files.download(f"{week}.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
skipped_frames

In [ ]:
skipped_plays